# ASBE - Automatic Stopping for Batch Experiments

> API details.

In [ ]:
#hide
from nbdev import *

In [ ]:
%nbdev_default_export core

Cells will be exported to asbe.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np

from modAL.models.base import BaseLearner
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from typing import Union, Optional
from copy import deepcopy
from pylift.eval import UpliftEval

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
%nbdev_export
def random_batch_sampling(classifier, X_pool, n2):
    "Randomly sample a batch from a pool of unlabaled samples"
    n_samples = X_pool.shape[0]
    query_idx = np.random.choice(range(n_samples), size=n2,replace=False)
    return X_pool[query_idx], query_idx

def uncertainty_batch_sampling(classifier, X_pool, n2):
    "Select the top $n_2$ most uncertain units"
    if classifier.estimator._most_recent_predicted is True:
        # Calculate variance based on predicted
        if classifier.estimator.y1_preds.shape[0] <= 1 or \
        len(classifier.estimator.y1_preds.shape) <= 1:
            raise Exception("Not possible to calculate uncertainty when dimensions <=1 ")
        ite_vars = np.var(classifier.estimator.y1_preds - classifier.estimator.y0_preds, axis=1)
        query_idx = np.argsort(ite_vars)[-n2:][::-1]
        return X_pool[query_idx], query_idx
        

In [ ]:
%nbdev_export
estimator_type = ClassifierMixin
class ASLearner(BaseLearner):
    """A(ctively)S(topping)Learner class for automatic stopping in batch-mode AL"""
    def __init__(self,
                 estimator: estimator_type=None, 
                 query_strategy=None,
                 assignment_fc=None,
                 X_training: np.ndarray = None,
                 t_training: np.ndarray = None,
                 y_training: np.ndarray = None,
                 X_pool: np.ndarray = None
                ) -> None:
        self.estimator = estimator
        self.query_strategy = query_strategy
        self.assignment_fc = assignment_fc
        self.X_training = X_training
        self.y_training = y_training
        self.t_training = t_training
        self.X_pool     = X_pool
        
    def _add_queried_data_class(self, X, t, y):
        self.X_training = np.vstack((self.X_training, X))
        self.t_training = np.concatenate((self.t_training, t))
        self.y_training = np.concatenate((self.y_training, y))
    
    def _update_estimator_values(self):
        self.estimator.__dict__.update(X_training = self.X_training,
                               y_training  =        self.y_training,
                               t_training  =        self.t_training,
                               X_test      =        self.X_pool)

    def teach(self, X_new, t_new, y_new):
        """Teaching new instances to the estimator selected bu the query_strategy
        
        If no `assignment_fc` is added, all selected samples are used
        If assignment function is added, only those instances are used, where
        $\hat{T} = T$
        """
        if self.assignment_fc is None:
            self._add_queried_data_class(X_new, t_test, y_test)
            self.estimator.fit()
    
    def fit(self):
        self._update_estimator_values()
        self.estimator.fit()
        
    def predict(self, X=None, **kwargs):
        if self.X_pool is not None:
            X = self.X_pool
        elif X is None:
            raise Exception("You need to supply an unlabeled pool of instances (with shape (-1,{}))".format(self.X_training.shape[1]))
        self.preds = self.estimator.predict(X, **kwargs)
        return self.preds
    
    def score(self, preds=None, y_true=None, t_true=None, metric = "Qini"):
        """
        Scoring the predictions - either ITE or observed outcomes are needed.
        
        If observed outcomes are provided, the accompanying treatments are also needed.
        """
        if metric == "Qini":
            upev = UpliftEval(t_true, y_true, self.preds[0] if preds is None else preds)
            self.scores = upev
        return self.scores.q1_aqini

In [ ]:
%nbdev_export
class ITEEstimator(BaseEstimator):
    """ Class for building a naive estimator for ITE estimation
    """
    def __init__(self,
                 model: estimator_type = None,
                 two_model: bool = False,
                 **kwargs
                ) -> None:
        self.model = model
        self.two_model = two_model

    def fit(self,X_training: np.ndarray = None,
                 t_training: np.ndarray = None,
                 y_training: np.ndarray = None,
                 X_test: np.ndarray = None):
        if X_training is not None:
            self.X_training = X_training
            self.y_training = y_training
            self.t_training = t_training
            self.X_test = X_test
        self.N_training = self.X_training.shape[0]
        # if "N_training" not in self.__dict__:
        #     self.N_training = self.X_training.shape[0]
        if self.two_model:
            self.m1 = deepcopy(self.model)
            control_ix = np.where(self.t_training == 0)[0]
            self.model.fit(self.X_training[control_ix,:],
                           self.y_training[control_ix])
            self.m1.fit(self.X_training[-control_ix,:],
                        self.y_training[-control_ix])
        else:
            self.model.fit(np.hstack((self.X_training,
                                      self.t_training.reshape((self.N_training, -1)))),
                           self.y_training)
        self._most_recent_predicted = False
            
    def _predict_without_proba(self, model, X, **kwargs):
        return model.predict(X,
            return_mean = kwargs["return_mean"] if "return_mean" in kwargs else True)
            
    def predict(self, X=None, **kwargs):
        self._most_recent_predicted = True
        if X is None:
            X = self.X_test
        N_test = X.shape[0]
        try:
            if self.two_model:
                self.y1_preds = self.m1.predict_proba(X)[:,1]
                self.y0_preds = self.model.predict_proba(X)[:,1]
            else:
                
                self.y1_preds = self.model.predict_proba(
                                    np.hstack((X,
                                    np.ones(N_test).reshape(-1,1))))[:,1]
                self.y0_preds = self.model.predict_proba(
                    np.hstack((X,
                               np.zeros(N_test).reshape(-1,1))))[:,1]
        except AttributeError:
            if type(self.model) is XBART:
                if self.two_model:
                    self.y1_preds = self._predict_without_proba(self.m1, X, **kwargs)
                    self.y0_preds = self._predict_without_proba(self.model, X, **kwargs)
                else: 
                    self.y1_preds = self._predict_without_proba(self.model,
                             np.hstack((X,
                             np.ones(N_test).reshape(-1,1))), **kwargs)
                    self.y0_preds = self._predict_without_proba(self.model,
                        np.hstack((X,
                                   np.zeros(N_test).reshape(-1,1))), **kwargs)
        return self.y1_preds - self.y0_preds, self.y1_preds, self.y0_preds

In [ ]:
X = np.random.normal(size = 1000).reshape((500,2))
t = np.random.binomial(n = 1, p = 0.5, size = 500)
y = np.random.binomial(n = 1, p = 1/(1+np.exp(X[:, 1]*2 + t*3)))
X_test = np.random.normal(size = 200).reshape((100,2))
t_test = np.random.binomial(n = 1, p = 0.5, size = 100)
y_test = np.random.binomial(n = 1, p = 1/(1+np.exp(X_test[:, 1]*2 + t_test*3)))
a = ITEEstimator(LogisticRegression(solver="lbfgs"), two_model = True)
a.fit(X, t, y)
assert type(a.model) == LogisticRegression  # test assigning a model
assert a.X_training.shape  == (500,2)       # test data passing for class
assert a.model.intercept_ is not None

In [ ]:
# a = ITEEstimator(RandomForestClassifier(), X, t, y, X_test, two_model = False )

In [ ]:
asl = ASLearner(estimator = ITEEstimator(model = RandomForestClassifier()), 
         query_strategy=random_batch_sampling,
         X_training=X,t_training=t,y_training=y,X_pool=X_test)
asl.fit()
ite_pred, y1_pred, y0_pred = asl.predict()
X_sel, query_sel = asl.query(asl.X_pool, n2=10)
assert ite_pred.shape[0] == 100
assert X_sel.shape       == (10,2)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
from xbart import XBART

In [ ]:
n_train = 100
p       = 5
n_test  = 1000
n2      = 20
X_train = np.random.normal(size = n_train*p).reshape((n_train,p))
t_train = np.random.binomial(n = 1, p = 0.5, size = n_train)
y_train = np.random.binomial(n = 1, p = 1/(1+np.exp(-1*(X_train[:, 1]*2 + t_train*3))))
X_test = np.random.normal(size = n_test*p).reshape((n_test,p))
t_test = np.random.binomial(n = 1, p = 0.5, size = n_test)
y_test = np.random.binomial(n = 1, p = 1/(1+np.exp(-1*(X_test[:, 1]*2 + t_test*3))))
asl = ASLearner(estimator = ITEEstimator(model = XBART(),two_model=False), 
         query_strategy=uncertainty_batch_sampling,
         X_training = X_train,
         t_training = t_train,
         y_training = y_train,
         X_pool     = X_test)
asl.fit()
p_ite, p_y1, p_y0 = asl.predict(asl.X_pool, return_mean=False)
print("Qini before AL: {}".format(asl.score(preds=np.mean(p_ite, axis=1),
                                            y_true=y_test, t_true=t_test)))
qini_vals = []
for _ in range(10):
    X_query, ix = asl.query(asl.X_pool, n2=n2)
    asl.teach(X_query, t_test[ix], y_test[ix])
    asl.X_pool = np.delete(asl.X_pool,ix, axis=0)
    t_test     = np.delete(t_test,ix, axis=0) 
    y_test     = np.delete(y_test,ix, axis=0) 
    p_ite, p_y1, p_y0 = asl.predict(asl.X_pool, return_mean=False)
    qini_vals.append(asl.score(preds=np.mean(p_ite, axis=1), y_true=y_test, t_true=t_test))
    print("Qini after round {} of AL: {}".format(_,qini_vals[_]))

Qini before AL: 0.006331193823948636
Qini after round 0 of AL: -0.006114237547775313
Qini after round 1 of AL: -0.010276294377066956
Qini after round 2 of AL: -0.009734900557350627
Qini after round 3 of AL: -0.01622029908854848
Qini after round 4 of AL: -0.01797305050101299
Qini after round 5 of AL: -0.019526769309971718
Qini after round 6 of AL: -0.02631464190247936
Qini after round 7 of AL: -0.031567887560428345
Qini after round 8 of AL: -0.03593669208558153
Qini after round 9 of AL: -0.03529111066978927
